# 🔧 Lab 02: Agent with Tools

**Goal:** Learn to create and use tools for AI agents.

**Time:** 60 minutes

---

## What you'll learn:
- How to define custom tools
- How agents choose the right tool
- Best practices for tool design

---

In [ ]:
#@title 🔧 Setup
!pip install -q google-generativeai smolagents litellm

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from smolagents import CodeAgent, tool
from smolagents.models import LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)

print("✅ Setup OK")

## Part 1: What is a Tool?

A **tool** is a function that an agent can call.

Examples:
- 🌤️ Get weather
- 🔍 Search database
- 📧 Send email
- 🧮 Calculate value

### Tool Anatomy:
```python
@tool
def my_tool(param1: str, param2: int) -> str:
    """
    Description of what the tool does.  <-- Agent reads this!
    
    Args:
        param1: Description of parameter 1
        param2: Description of parameter 2
    """
    # Logic
    return "result"
```

In [ ]:
#@title 🔧 First tool: Calculator

@tool
def calculate(expression: str) -> str:
    """
    Calculates a mathematical expression.
    
    Args:
        expression: Mathematical expression to calculate (e.g., "2 + 2 * 3")
    """
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Test the tool directly
print(calculate("15 * 7 + 23"))

In [ ]:
#@title 🤖 Agent with calculator

calc_agent = CodeAgent(
    tools=[calculate],
    model=model,
    max_steps=3
)

# Agent can now calculate precisely
result = calc_agent.run("What is 15% of 1240?")
print(result)

## Part 2: Multiple Tools

In [ ]:
#@title 🔧 Defining multiple tools

@tool
def get_weather(city: str) -> str:
    """
    Gets the current weather for a given city.
    
    Args:
        city: Name of the city (e.g., "Prague", "London", "New York")
    """
    # Demo database (in reality you'd call a weather API)
    weather_data = {
        "prague": {"temp": 12, "condition": "☀️ sunny"},
        "london": {"temp": 10, "condition": "🌤️ partly cloudy"},
        "new york": {"temp": 14, "condition": "⛅ cloudy"},
        "vienna": {"temp": 11, "condition": "🌧️ rain"},
    }
    
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        return f"{city}: {data['temp']}°C, {data['condition']}"
    return f"Weather for {city} is not available"

@tool
def get_exchange_rate(from_currency: str, to_currency: str) -> str:
    """
    Gets the exchange rate between currencies.
    
    Args:
        from_currency: Source currency (e.g., "EUR", "USD", "GBP")
        to_currency: Target currency
    """
    # Demo rates
    rates = {
        ("EUR", "USD"): 1.08,
        ("EUR", "GBP"): 0.86,
        ("USD", "EUR"): 0.93,
        ("GBP", "EUR"): 1.16,
    }
    
    key = (from_currency.upper(), to_currency.upper())
    if key in rates:
        return f"1 {from_currency.upper()} = {rates[key]} {to_currency.upper()}"
    return f"Rate {from_currency}/{to_currency} is not available"

print("✅ Tools defined:")
print("   - calculate")
print("   - get_weather")
print("   - get_exchange_rate")

In [ ]:
#@title 🤖 Multi-tool Agent

multi_agent = CodeAgent(
    tools=[calculate, get_weather, get_exchange_rate],
    model=model,
    max_steps=5,
    verbosity_level=1  # Shows which tools were used
)

print("Agent has access to:")
for t in multi_agent.tools.values():
    print(f"   - {t.name}")
print()

In [ ]:
#@title ▶️ Test: Complex task

task = """
I'm planning a trip to Prague. Tell me:
1. What's the weather like there?
2. How many USD will I get for 100 EUR?
"""

print(f"📋 Task:\n{task}")
print("="*50)

result = multi_agent.run(task)

print("="*50)
print(f"\n✅ Answer:\n{result}")

## Part 3: Tool with Logic

In [ ]:
#@title 🔧 More complex tool: Product Search

import json

# Simulated product database
PRODUCTS = [
    {"id": 1, "name": "Laptop Dell XPS 15", "price": 1299, "category": "electronics", "stock": 5},
    {"id": 2, "name": "iPhone 15 Pro", "price": 999, "category": "electronics", "stock": 12},
    {"id": 3, "name": "Sony WH-1000XM5", "price": 349, "category": "electronics", "stock": 8},
    {"id": 4, "name": "Office Chair Ergonomic", "price": 299, "category": "furniture", "stock": 3},
    {"id": 5, "name": "Standing Desk", "price": 449, "category": "furniture", "stock": 0},
]

@tool
def search_products(query: str, max_price: int = 10000) -> str:
    """
    Searches for products in the catalog.
    
    Args:
        query: Search term (part of name or category)
        max_price: Maximum price in EUR (default: 10000)
    """
    results = []
    query_lower = query.lower()
    
    for product in PRODUCTS:
        name_match = query_lower in product["name"].lower()
        category_match = query_lower in product["category"].lower()
        price_ok = product["price"] <= max_price
        
        if (name_match or category_match) and price_ok:
            results.append(product)
    
    if not results:
        return "No products found."
    
    output = f"Found {len(results)} products:\n"
    for p in results:
        stock_status = "✅ in stock" if p["stock"] > 0 else "❌ sold out"
        output += f"- {p['name']}: ${p['price']} ({stock_status})\n"
    
    return output

@tool
def check_stock(product_name: str) -> str:
    """
    Checks availability of a specific product.
    
    Args:
        product_name: Product name (doesn't have to be exact)
    """
    for product in PRODUCTS:
        if product_name.lower() in product["name"].lower():
            if product["stock"] > 0:
                return f"{product['name']}: {product['stock']} units in stock"
            else:
                return f"{product['name']}: SOLD OUT"
    return f"Product '{product_name}' not found"

# Test
print(search_products("electronics", 500))

In [ ]:
#@title 🛒 E-commerce Agent

shop_agent = CodeAgent(
    tools=[search_products, check_stock, calculate],
    model=model,
    max_steps=5,
    system_prompt="""You are an e-shop assistant. You help customers find products.
    Always:
    - Respond friendly
    - Include prices
    - Inform about availability
    """
)

result = shop_agent.run("I'm looking for headphones under $400. What do you have?")
print(result)

In [ ]:
#@title ▶️ Test: Complex scenario

result = shop_agent.run("""
I need to set up a home office. 
My budget is $800.
What would you recommend?
""")
print(result)

## Part 4: Best Practices

### ✅ Good tools:
1. **Clear description** - agent must understand what the tool does
2. **Specific parameters** - with types and descriptions
3. **Meaningful output** - structured response
4. **Error handling** - tool doesn't crash

### ❌ Bad tools:
1. Vague description
2. Too many parameters
3. Tool does too many things
4. Doesn't return useful information

In [ ]:
#@title ❌ Example of a BAD tool

# ❌ Bad tool - unclear description, missing types
@tool
def do_stuff(x):
    """Does stuff with x"""
    return str(x * 2)

# Agent doesn't know when and how to use this tool!

In [ ]:
#@title ✅ Example of a GOOD tool

@tool
def convert_temperature(value: float, from_unit: str, to_unit: str) -> str:
    """
    Converts temperature between units (Celsius, Fahrenheit, Kelvin).
    
    Args:
        value: Numeric temperature value
        from_unit: Source unit ("C", "F", or "K")
        to_unit: Target unit ("C", "F", or "K")
    """
    # First convert to Celsius
    if from_unit.upper() == "F":
        celsius = (value - 32) * 5/9
    elif from_unit.upper() == "K":
        celsius = value - 273.15
    else:
        celsius = value
    
    # Then to target unit
    if to_unit.upper() == "F":
        result = celsius * 9/5 + 32
    elif to_unit.upper() == "K":
        result = celsius + 273.15
    else:
        result = celsius
    
    return f"{value}°{from_unit.upper()} = {result:.2f}°{to_unit.upper()}"

# Test
print(convert_temperature(100, "C", "F"))

---

## 🏋️ Exercise: Create your own toolset

**Task:** Create an agent for customer support with tools:

1. `search_faq(query)` - searches the FAQ
2. `create_ticket(summary, priority)` - creates a support ticket
3. `check_order_status(order_id)` - checks order status

In [ ]:
#@title ✏️ Your solution

# FAQ database
FAQ = {
    "refund": "Refunds are possible within 14 days of delivery.",
    "shipping": "Standard delivery takes 3-5 business days.",
    "payment": "We accept Visa, Mastercard, and PayPal.",
}

# Orders
ORDERS = {
    "ORD-001": {"status": "shipped", "tracking": "SK123456"},
    "ORD-002": {"status": "processing"},
    "ORD-003": {"status": "delivered"},
}

# TODO: Implement the tools

@tool
def search_faq(query: str) -> str:
    """
    # ADD DOCSTRING HERE
    """
    # ADD IMPLEMENTATION HERE
    pass

@tool
def create_ticket(summary: str, priority: str) -> str:
    """
    # ADD DOCSTRING HERE
    """
    # ADD IMPLEMENTATION HERE
    pass

@tool
def check_order_status(order_id: str) -> str:
    """
    # ADD DOCSTRING HERE
    """
    # ADD IMPLEMENTATION HERE
    pass


# Create the agent
support_agent = CodeAgent(
    tools=[search_faq, create_ticket, check_order_status],
    model=model,
    max_steps=5
)

# Test:
# result = support_agent.run("Where is my order ORD-001?")
# print(result)

---

## 📝 Reflection

1. **Why is a good docstring important?**

2. **How does an agent decide which tool to use?**

3. **When does it make sense to split functionality into multiple tools?**

---

## ➡️ Next Step

In the next lab, we'll learn **RAG (Retrieval-Augmented Generation)** - how to give an agent access to documents.

→ **[03_RAG_Agent.ipynb](03_RAG_Agent.ipynb)**